# Example notebook for torch_mixture package

In [1]:
from torch_mixture import torch_mixture
import torch.distributions as tdist

In [2]:
# torch_mixture supports most of the univariate torch distributions
torch_mixture.available_distributions()

[torch.distributions.normal.Normal,
 torch.distributions.log_normal.LogNormal,
 torch.distributions.gamma.Gamma,
 torch.distributions.gumbel.Gumbel,
 torch.distributions.cauchy.Cauchy,
 torch.distributions.studentT.StudentT,
 torch.distributions.laplace.Laplace,
 torch.distributions.beta.Beta,
 torch.distributions.exponential.Exponential,
 torch.distributions.fishersnedecor.FisherSnedecor,
 torch.distributions.half_cauchy.HalfCauchy,
 torch.distributions.half_normal.HalfNormal,
 torch.distributions.pareto.Pareto,
 torch.distributions.poisson.Poisson,
 torch.distributions.bernoulli.Bernoulli,
 torch.distributions.uniform.Uniform]

## Example: fitting two log normal models to FOSL2 fragment length data

In [3]:
from datetime import datetime

In [4]:
import pandas as pd
fosl2 = pd.read_csv('/home/katie/torch_mixture/examples/FOSL2_frag_lengths.txt',
                   header=None).values.ravel()

### Fitting the models

In [5]:
print(datetime.now())

# Mixture object requires a 1D numpy array and a list of torch distributions
# Optionally (recommended): you can initialize with k means clustering rather than random initialization
mix = torch_mixture.Mixture(fosl2, [tdist.LogNormal, tdist.LogNormal], cluster_initialize=True)

# fit takes two parameters:
# learning_rate: learning rate used by Adam optimizer, e.g. 1e-1
# n_iter: number of iterations to run the optimizer, e.g. 500
mix.fit(1e-1, 500)

print(datetime.now())

2021-10-17 23:21:52.588723
at initialization negative log-likelihood is 149,468,816.0
negative log-likelihood is 141,743,360.0
relative gradients are [0.00695877 0.00804878 0.01465199 0.00069342]
weights: [0.38 0.62]
2021-10-17 23:22:10.009233


### Using the mixture models

In [6]:
# get fitted models, weights, and parameters
dists = mix.calc_distributions()
weights = mix.weights.cpu()

print("Model 1: {}\nWeight: {}\nMean: {}\nStandard deviation: {}\n\n\
Model 2: {}\nWeight: {}\nMean: {}\nStandard deviation: {}" \
      .format(str(dists[0]), mix.weights[0], dists[0].mean, dists[0].stddev,
              str(dists[1]), mix.weights[1], dists[1].mean, dists[1].stddev))

Model 1: LogNormal()
Weight: 0.3831370770931244
Mean: 297.19354248046875
Standard deviation: 171.36109924316406

Model 2: LogNormal()
Weight: 0.6166378259658813
Mean: 151.3415069580078
Standard deviation: 115.76274108886719


In [14]:
import torch

# predict some example observations
mix.predict(torch.tensor([300, 350, 1000]))

tensor([[0.6523, 0.3482],
        [0.6977, 0.3024],
        [0.8063, 0.1939]], device='cuda:0')